In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("base propre.csv", sep=",")
data

,Date,Jour,Chaine,Heure_prgm1,Titre_prgm1,Type_prgm1,Duree_prgm1,Nbre_episodes_prgm1,Age_conseille_prgm1,Heure_prgm2,Titre_prgm2,Type_prgm2,Duree_prgm2,Nbre_episodes_prgm2,Age_conseille_prgm2,Part_de_marche,Annee,Mois,Week end
0,08 octobre 2014,mercredi,TF1,20.55,Blacklist,Serie TV,110.0,2.0,-10 ans,22.40,Arrow,Serie TV,100.0,3.0,-10 ans,24.0,2014,octobre,0
1,08 octobre 2014,mercredi,France 2,20.50,Les hommes de l'ombre,Serie TV,120.0,2.0,Tout public,22.40,Un soir à la tour Eiffel,Magazine,100.0,0.0,Tout public,13.0,2014,octobre,0
2,08 octobre 2014,mercredi,France 3,20.45,Des racines et des ailes,Magazine,115.0,0.0,Tout public,23.36,Le comte Ory,Divertissement,144.0,0.0,Tout public,12.0,2014,octobre,0
3,08 octobre 2014,mercredi,Canal +,20.55,Diana,Film,110.0,0.0,Tout public,22.45,9 mois ferme,Film,80.0,0.0,Tout public,NaN,2014,octobre,0
4,08 octobre 2014,mercredi,France 5,20.40,La maison France 5,Magazine,60.0,0.0,Tout public,21.40,"Silence, ça pousse !",Magazine,49.0,0.0,Tout public,2.0,2014,octobre,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48341,31 décembre 2019,NaN,Gulli,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2019,décembre,0
48342,31 décembre 2019,NaN,C Star,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2019,décembre,0
48343,31 décembre 2019,NaN,Cherie 25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2019,décembre,0
48344,31 décembre 2019,NaN,RMC Story,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2019,décembre,0


# Suppression des NA

In [3]:
data.isna().sum()

Date                       0
Jour                     575
Chaine                     6
Heure_prgm1              575
Titre_prgm1             1264
Type_prgm1              1266
Duree_prgm1              575
Nbre_episodes_prgm1      575
Age_conseille_prgm1      575
Heure_prgm2              575
Titre_prgm2              969
Type_prgm2               974
Duree_prgm2              575
Nbre_episodes_prgm2      575
Age_conseille_prgm2      575
Part_de_marche         16227
Annee                      0
Mois                       0
Week end                   0
dtype: int64

In [4]:
data.dropna(axis=0, subset=['Jour','Chaine','Part_de_marche','Type_prgm1','Type_prgm2'],inplace=True)

In [5]:
data.isna().sum()

Date                   0
Jour                   0
Chaine                 0
Heure_prgm1            0
Titre_prgm1            0
Type_prgm1             0
Duree_prgm1            0
Nbre_episodes_prgm1    0
Age_conseille_prgm1    0
Heure_prgm2            0
Titre_prgm2            0
Type_prgm2             0
Duree_prgm2            0
Nbre_episodes_prgm2    0
Age_conseille_prgm2    0
Part_de_marche         0
Annee                  0
Mois                   0
Week end               0
dtype: int64

In [6]:
data

,Date,Jour,Chaine,Heure_prgm1,Titre_prgm1,Type_prgm1,Duree_prgm1,Nbre_episodes_prgm1,Age_conseille_prgm1,Heure_prgm2,Titre_prgm2,Type_prgm2,Duree_prgm2,Nbre_episodes_prgm2,Age_conseille_prgm2,Part_de_marche,Annee,Mois,Week end
0,08 octobre 2014,mercredi,TF1,20.55,Blacklist,Serie TV,110.0,2.0,-10 ans,22.40,Arrow,Serie TV,100.0,3.0,-10 ans,24.0,2014,octobre,0
1,08 octobre 2014,mercredi,France 2,20.50,Les hommes de l'ombre,Serie TV,120.0,2.0,Tout public,22.40,Un soir à la tour Eiffel,Magazine,100.0,0.0,Tout public,13.0,2014,octobre,0
2,08 octobre 2014,mercredi,France 3,20.45,Des racines et des ailes,Magazine,115.0,0.0,Tout public,23.36,Le comte Ory,Divertissement,144.0,0.0,Tout public,12.0,2014,octobre,0
4,08 octobre 2014,mercredi,France 5,20.40,La maison France 5,Magazine,60.0,0.0,Tout public,21.40,"Silence, ça pousse !",Magazine,49.0,0.0,Tout public,2.0,2014,octobre,0
5,08 octobre 2014,mercredi,M6,20.50,Le meilleur pâtissier,Divertissement,110.0,0.0,Tout public,22.40,Le meilleur pâtissier : à vos fourneaux !,Divertissement,100.0,0.0,Tout public,12.0,2014,octobre,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47740,09 décembre 2019,lundi,6ter,21.05,Rupture pour tous,Film,110.0,0.0,-10 ans,22.55,Hors de prix,Film,120.0,0.0,Tout public,0.0,2019,décembre,0
47741,09 décembre 2019,lundi,TF1 Series,21.00,Un festival pour Noël,Telefilm,100.0,0.0,Tout public,22.40,La clé d'un Noël réussi,Telefilm,105.0,0.0,Tout public,2.0,2019,décembre,0
47743,09 décembre 2019,lundi,RMC Story,20.55,Enquête prioritaire,Magazine,80.0,0.0,Tout public,23.05,"La Poste, une logistique XXL",Documentaire,65.0,0.0,Tout public,0.0,2019,décembre,0
47744,09 décembre 2019,lundi,RMC Decouverte,20.55,Le convoi de l'extrême : le choc des titans,Divertissement,90.0,4.0,Tout public,Suite prgm1,Le convoi de l'extrême : le choc des titans,Divertissement,0.0,4.0,Tout public,1.0,2019,décembre,0


In [7]:
data.to_csv("Donnees_sans_na.csv",index=False)

# Recodage par label encoder

In [8]:
from sklearn.preprocessing import (OneHotEncoder,
                                   LabelEncoder,
                                   StandardScaler,
                                   Normalizer,
                                   MinMaxScaler,
                                  )

In [9]:
#data['Jour'] = data['Jour'].astype('category')
#data['Jour'].cat.categories = [7,4,1,2,3,6,5] 
#data['Jour'].astype('float')
#Fonctionne mais n'utilise pas label encoder

In [10]:
gestion_jour = LabelEncoder()
gestion_mois = LabelEncoder()
data['Jour'] = gestion_jour.fit_transform(data["Jour"].astype(str))
data['Mois'] = gestion_mois.fit_transform(data["Mois"].astype(str))

In [11]:
gestion_heure_prgm1 = LabelEncoder()
gestion_heure_prgm2 = LabelEncoder()
data['Heure_prgm1'] = gestion_heure_prgm1.fit_transform(data["Heure_prgm1"].astype(str))
data['Heure_prgm2'] = gestion_heure_prgm2.fit_transform(data["Heure_prgm2"].astype(str))

In [12]:
gestion_age_prgm1 = LabelEncoder()
gestion_age_prgm2 = LabelEncoder()
data['Age_conseille_prgm1'] = gestion_age_prgm1.fit_transform(data["Age_conseille_prgm1"].astype(str))
data['Age_conseille_prgm2'] = gestion_age_prgm2.fit_transform(data["Age_conseille_prgm2"].astype(str))

In [13]:
reste = data[["Jour","Week end","Mois","Annee","Heure_prgm1","Duree_prgm1", "Nbre_episodes_prgm1","Age_conseille_prgm1",
              "Heure_prgm2","Duree_prgm2", "Nbre_episodes_prgm2","Age_conseille_prgm2", "Part_de_marche"]].values

# Recodage par One Out Encoder

In [14]:
gestion_chaine = OneHotEncoder()
chaine_ohe = gestion_chaine.fit_transform(data["Chaine"]
                                  .astype(str)
                                  .values
                                  .reshape(-1, 1)
                                 )

In [15]:
gestion_type_prgm1 = OneHotEncoder()
gestion_type_prgm2 = OneHotEncoder()
type_prgm1_ohe = gestion_type_prgm1.fit_transform(data["Type_prgm1"]
                                  .astype(str)
                                  .values
                                  .reshape(-1, 1)
                                 )
type_prgm2_ohe = gestion_type_prgm2.fit_transform(data["Type_prgm2"]
                                  .astype(str)
                                  .values
                                  .reshape(-1, 1)
                                 )

# Séparation variables explicatives et à expliquer

In [16]:
X = np.concatenate((chaine_ohe.toarray(),type_prgm2_ohe.toarray(),reste), axis=1)

In [17]:
X.shape

(30981, 43)

In [18]:
X

array([[0.0, 0.0, 0.0, ..., 3.0, 0, '24.4'],
       [0.0, 0.0, 0.0, ..., 0.0, 3, '13.1'],
       [0.0, 0.0, 0.0, ..., 0.0, 3, '12'],
       ...,
       [0.0, 0.0, 0.0, ..., 0.0, 3, '0.5'],
       [0.0, 0.0, 0.0, ..., 4.0, 3, '1.8'],
       [0.0, 0.0, 0.0, ..., 0.0, 3, '1.2']], dtype=object)

In [19]:
y = type_prgm1_ohe.toarray()

In [20]:
y.shape

(30981, 8)

# Séparation des échantillons

In [21]:
from sklearn.model_selection import (train_test_split)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Normalisation par Standard Scaler

In [27]:
ss = StandardScaler()
mm = MinMaxScaler()

In [28]:
X_ess = ss.fit_transform(X_train)
X_emm = mm.fit_transform(X_train)
#A revoir

ValueError: could not convert string to float: 'Suite'